# Package Imports

In [66]:
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd # Data manipulation package, especially for tabular data, most downladed public python package
import re # Regular expression operations (string matching and manipulation)

from typing import Any, Optional, Iterable, List


# Filepaths

In [67]:
fp_biomarkers = "Data/LS Biomarkers Timeseries.csv" # Read in the biomarker time series data
fp_biomarkersLong = "Data/LLM_Biomarkers_Long.csv" # Write out the long format data
fp_biomarkersWide = "Data/LLM_Biomarkers_Wide.csv" # Write out the wide format data

fp_microbiome = "Data/LS Gut Microbiome.csv" # Read in the microbiome data
fp_microbiomeLong = "Data/LLM_Gut_Microbiome_Long.csv" # Write out the long format data
fp_microbiomeWide = "Data/LLM_Gut_Microbiome_Wide.csv" # Write out the wide format data
fp_microbiomeSummary = "Data/LLM_Gut_Microbiome_Summary.csv" # Write out the summary data

# Biomarker Time Series Data 

In [68]:
df_biomarkers = pd.read_csv(fp_biomarkers)
df_biomarkers

,Date collected,Units,Good Range,5/1/1993,11/27/1996,5/8/1997,4/15/1998,4/1/1999,10/6/1999,5/7/2001,...,2/5/2025,2/12/2025,2/28/2025,5/20/2025,6/13/2025,Low,High,Average,Number,Unnamed: 341
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,YFH/UCSD,UCSD,UCSD,UCSD,UCSD,of tests since '07,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Weight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,201.0,...,178,179,179.6,182.5,180.8,166.1,202.0,185.6,322.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Blood Tests,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Butyrate,mg/mL,0.8-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,NaN,NaN,NaN,NaN,0.3,4.3,1.3,184.0,NaN
234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create Long Format

### Helper Functions

In [69]:
def is_blank(x: Any) -> bool:
    """ Return True if `x` is effectively empty: None, NaN, or a string of only whitespace.
    Parameters-
    x : Any, Value to check
    Returns-
    bool: True if `x` is None, NaN, or a blank/whitespace-only string; False otherwise."""
    
    return (x is None) or (pd.isna(x)) or (str(x).strip() == "")

def find_col(name_like: str, default: Optional[str] = None) -> Optional[str]:
    """ Return the FIRST column in global `cols` whose (lowercased, trimmed) text
    contains the (lowercased, trimmed) substring `name_like`.
    Parameters-
    name_like : str, Substring to look for in each column name.
    default : Optional[str], default None, Value to return if no match is found."""
    
    for c in cols:
        if name_like in str(c).strip().lower():
            return c
    return default

def find_exact(label: str) -> Optional[str]:
    """ Return an EXACT matching column from global `cols`. If not found,
    return a case-insensitive match on the whole label (after trimming).
    Parameters-
    label : str, Column label to match.
    Returns-
    Optional[str]: Matching column name from `cols`, or None if no match."""
    
    for c in cols:
        if c == label:
            return c
    low = label.lower()
    for c in cols:
        if str(c).strip().lower() == low:
            return c
    return None

def uniq_order(seq):
    """ Deduplicate while preserving first-seen order.
    - Converts items to trimmed strings.
    - Skips None/blank items.
    Parameters-
    seq : Iterable[Any], Sequence of values.
    Returns-
    List[str]: List of unique, trimmed, non-blank strings in their first-seen order. """
    
    seen = set()
    out = []
    for x in seq:
        if x is None: 
            continue
        s = str(x).strip()
        if not s or s in seen:
            continue
        seen.add(s)
        out.append(s)
    return out

def clean_label(x):
    """ Normalize a label/string:
    - Return None if blank (None/NaN/whitespace-only).
    - Replace non-breaking spaces (\\u00A0) with regular spaces.
    - Trim outer whitespace.

    Parameters-
    x : Any, Value to clean.
    Returns-
    Optional[str]: Cleaned string, or None if input is blank."""
    if is_blank(x):
        return None
    return str(x).replace("\u00a0", " ").strip()

### Mapping Dictionaries for Columns

In [70]:
# Mapping For biomarkeers to panels
dict_biomarkerToPanels = {
                            # Vitals
                            "Weight": "Vitals",

                            # LIPID Panel
                            "237": "LIPID Panel",
                            'HDL (High Density Lipoprotein) "good"': "LIPID Panel",
                            "VLDL Cholesterol": "LIPID Panel",
                            "IDL Chloresterol": "LIPID Panel",
                            "Sum Total Cholesterol": "LIPID Panel",
                            "Triglycerides": "LIPID Panel",
                            "UCSD LDL Particle # by NMR": "LIPID Panel",
                            "UCSD HDL Particle #": "LIPID Panel",
                            "UCSD Large VLDL Particle #": "LIPID Panel",
                            "UCSD Small LDL Particle #": "LIPID Panel",
                            "UCSD Large HDL Particle #": "LIPID Panel",
                            "UCSD VLDL Particle Size": "LIPID Panel",
                            "UCSD LDL Particle Size": "LIPID Panel",
                            "UCSD HDL Particle Size": "LIPID Panel",
                            "Real-LDL Size Pattern": "LIPID Panel",
                            "TG/HDL": "LIPID Panel",
                            "Chol/HDL": "LIPID Panel",
                            "Lp(a) Chloresterol": "LIPID Panel",
                            "APO A1": "LIPID Panel",
                            "APOLIPOPROTEIN-(B100)": "LIPID Panel",
                            "Ratio APO B100/AI": "LIPID Panel",

                            # Comprehensive Metabolic Panel – Electrolytes
                            "Sodium": "Comprehensive Metabolic Panel",
                            "Potassium": "Comprehensive Metabolic Panel",
                            "Chloride": "Comprehensive Metabolic Panel",
                            "Calcium": "Comprehensive Metabolic Panel",
                            "Phosporus": "Comprehensive Metabolic Panel",
                            "Magnesium": "Comprehensive Metabolic Panel",
                            "Zinc": "Comprehensive Metabolic Panel",
                            "Tot CO2": "Comprehensive Metabolic Panel",
                            "Bicarbonate": "Comprehensive Metabolic Panel",
                            "Anion Gap": "Comprehensive Metabolic Panel",
                            
                            # Comprehensive Metabolic Panel – remainder
                            "Total Protein": "Comprehensive Metabolic Panel",
                            "Albumin": "Comprehensive Metabolic Panel",
                            "Globulin": "Comprehensive Metabolic Panel",
                            "A/G ratio": "Comprehensive Metabolic Panel",
                            "GGTP": "Comprehensive Metabolic Panel",
                            "LDH": "Comprehensive Metabolic Panel",
                            "ALT": "Comprehensive Metabolic Panel",
                            "AST": "Comprehensive Metabolic Panel",
                            "AST/ALT ratio": "Comprehensive Metabolic Panel",
                            "Alkaline phosphatase": "Comprehensive Metabolic Panel",
                            "Total Bilirubin": "Comprehensive Metabolic Panel",
                            "Direct Bilirubin": "Comprehensive Metabolic Panel",

                            # Kidney
                            "Blood Urea Nitrogen (BUN)": "Kidney",
                            "Creatinine": "Kidney",
                            "BUN/Creatinine Ratio": "Kidney",
                            "Uric Acid": "Kidney",

                            # Glucose/Insulin Panel
                            "Fasting Glucose": "Glucose/Insulin Panel",
                            "Insulin": "Glucose/Insulin Panel",
                            "Hemoglobin A1C": "Glucose/Insulin Panel",
                            "HOMA-IR (insulin resistance)": "Glucose/Insulin Panel",

                            # Hormone Panel
                            "Free Testosterone": "Hormone Panel",
                            "Testosterone % Free": "Hormone Panel",
                            "Total Testosterone": "Hormone Panel",
                            "Sex Hormone Binding Globulin (SHBG)": "Hormone Panel",
                            "Estradiol": "Hormone Panel",
                            "DHEA-blood": "Hormone Panel",
                            "Cortisol Random": "Hormone Panel",
                            "TSH": "Hormone Panel",
                            "Total T3": "Hormone Panel",
                            "Thyroxine (T4)": "Hormone Panel",

                            # Cancer Panel
                            "PSA Screen": "Cancer Panel",
                            "CA-19-9 (Pancreatic Cancer)": "Cancer Panel",
                            "CEA": "Cancer Panel",

                            # Iron Panel
                            "Iron": "Iron Panel",
                            "Ferritin": "Iron Panel",
                            "Fibrinogen": "Iron Panel",
                            "TIBC": "Iron Panel",
                            "UIBC": "Iron Panel",
                            "Iron saturation": "Iron Panel",

                            # CBC – Red blood cell details
                            "RBC-": "CBC: Red Blood Cell Details",
                            "HGB-": "CBC: Red Blood Cell Details",
                            "HCT": "CBC: Red Blood Cell Details",
                            "PLT-": "CBC: Red Blood Cell Details",
                            "MCV-": "CBC: Red Blood Cell Details",
                            "MCH": "CBC: Red Blood Cell Details",
                            "MCHC-": "CBC: Red Blood Cell Details",
                            "RDW": "CBC: Red Blood Cell Details",
                            "MPV": "CBC: Red Blood Cell Details",

                            # CBC – White blood cell details
                            "WBC-": "CBC: White Blood Cell Details",
                            "NEU %": "CBC: White Blood Cell Details",
                            "LYM %": "CBC: White Blood Cell Details",
                            "EOS %": "CBC: White Blood Cell Details",
                            "MONO %": "CBC: White Blood Cell Details",
                            "BASO %": "CBC: White Blood Cell Details",
                            "Neutrophil Count": "CBC: White Blood Cell Details",
                            "Lymphocyte Count": "CBC: White Blood Cell Details",
                            "Monocyte Count": "CBC: White Blood Cell Details",
                            "Esoinophil Count": "CBC: White Blood Cell Details",
                            "Basophil Count": "CBC: White Blood Cell Details",

                            # Inflammation Panel
                            "High Sensitivity CRP (mg/L)": "Inflammation Panel",
                            "Lp-PLA2": "Inflammation Panel",
                            "IgG": "Inflammation Panel",
                            "IgA": "Inflammation Panel",
                            "IgM": "Inflammation Panel",

                            # Vitamins, Antioxidants, Amino Acid Panel
                            "Homocysteine": "Vitamins, Antioxidants, Amino Acid Panel",
                            "CoQ10": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Selenium": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin E Alpha-Tocopherol": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin E Gamma-Tocopherol": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Alpha Lipoic Acid": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin D, 25-Hydroxy": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Vitamin B12": "Vitamins, Antioxidants, Amino Acid Panel",
                            "Niacin (Vit B3)": "Vitamins, Antioxidants, Amino Acid Panel",

                            # Stool tests
                            "pH": "Stool tests",

                            # Cultured Bacteria Growth, Expected/Beneficial flora
                            "Bifidobacerium family": "Expected/Beneficial flora",
                            "E. coli spp.": "Expected/Beneficial flora",
                            "Lactobacillus spp": "Expected/Beneficial flora",
                            "Enterococcus spp.": "Expected/Beneficial flora",
                            "Bacteroides fraglis group": "Expected/Beneficial flora",
                            "Clostridium spp": "Expected/Beneficial flora",
                            "Bacteroides family": "Expected/Beneficial flora",

                            # Cultured Bacteria Growth, Commensal (Imbalanced) flora
                            "Alpha hemolytic strep": "Commensal (Imbalanced) flora",
                            "Hemolytic Escherichia coli": "Commensal (Imbalanced) flora",
                            "Pseudomonas chlororaphis group": "Commensal (Imbalanced) flora",
                            "Gamma Hemolytic Streptococcus": "Commensal (Imbalanced) flora",
                            "Hafnia alvei": "Commensal (Imbalanced) flora",
                            "Mucoid Escherichia coli": "Commensal (Imbalanced) flora",
                            "Beta strep, group B, strep. Agalactiae": "Commensal (Imbalanced) flora",
                            "Klebsiella oxytoca": "Commensal (Imbalanced) flora",
                            "Bacillus spp, pumilus": "Commensal (Imbalanced) flora",
                            "Comamonas testosteroni": "Commensal (Imbalanced) flora",
                            "Streptococcus anginosus": "Commensal (Imbalanced) flora",
                            "Cellulosimicrobium cellulans/funkeil/marinum": "Commensal (Imbalanced) flora",
                            "Lactococcus lactis": "Commensal (Imbalanced) flora",
                            "Staphylococcus aureus": "Commensal (Imbalanced) flora",
                            "Streptococcus salivarius": "Commensal (Imbalanced) flora",
                            "Streptococcus parasanguinis": "Commensal (Imbalanced) flora",
                            "Pseudomonas mendocina ": "Commensal (Imbalanced) flora",
                            "Actinomyces turicensis": "Commensal (Imbalanced) flora",
                            "Lysinibacillus spp": "Commensal (Imbalanced) flora",
                            "Streptococcus mutans": "Commensal (Imbalanced) flora",
                            "Streptococcus australis": "Commensal (Imbalanced) flora",
                            "Enterobacter asburiae": "Commensal (Imbalanced) flora",
                            "Pseudomonas stutzeri": "Commensal (Imbalanced) flora",
                            "Kocuria spp, salsicia, rhizophila, kristinae": "Commensal (Imbalanced) flora",
                            "Rothia dentocariosa/mucliaginosa": "Commensal (Imbalanced) flora",
                            "Pantoea spp": "Commensal (Imbalanced) flora",
                            "Pseudomonas citronellolis": "Commensal (Imbalanced) flora",
                            "Streptococcus mitis/oralis/cristatus": "Commensal (Imbalanced) flora",
                            "Staphylococcus haemolyticus": "Commensal (Imbalanced) flora",
                            "Serratia marcescens": "Commensal (Imbalanced) flora",
                            "Staphylococcus lugdunensis": "Commensal (Imbalanced) flora",
                            "Staphylococcus epidermidis": "Commensal (Imbalanced) flora",
                            "Actinomyces neuii": "Commensal (Imbalanced) flora",
                            "Streptomyces": "Commensal (Imbalanced) flora",
                            "Bacillus licheniformis": "Commensal (Imbalanced) flora",
                            "Corynebacterium falsenii": "Commensal (Imbalanced) flora",

                            # Dysbiotic bacteria
                            "Morganellla morganii": "Dysbiotic bacteria",
                            "Providencia stuartii": "Dysbiotic bacteria",
                            "Klebsiella pneumonia": "Dysbiotic bacteria",
                            "Providencia rettgeri": "Dysbiotic bacteria",
                            "Enterobacter cloacae": "Dysbiotic bacteria",
                            "Pseudomonas aeruginosa": "Dysbiotic bacteria",
                            "Bacillus cereus": "Dysbiotic bacteria",
                            "Citrobacter freundii complex": "Dysbiotic bacteria",
                            "Arcobacter butzieri": "Dysbiotic bacteria",

                            # Yeast
                            "Candida parapsilosis": "Yeast",
                            "Rhodotorula glutinis/mucilaginosa": "Yeast",
                            "Candida krusei/inconspicua/lambica": "Yeast",
                            "Saccharomyces cerevisiae/boulardi": "Yeast",
                            "Geotrichum spp.": "Yeast",
                            "Candida pararugosa": "Yeast",
                            "Candida lusitaniae": "Yeast",
                            "Candida zeylanoides": "Yeast",
                            "Trichosporon asahii": "Yeast",
                            "Trichosporon ovoides": "Yeast",
                            "Trichosporon montevideense": "Yeast",
                            "Exophiala dermatitidis": "Yeast",
                            "Cryptococcus spp, not neoformans": "Yeast",
                            "Cryptococcus diffuens": "Yeast",
                            "Candida orthopsilosis": "Yeast",

                            # Gut Inflammation
                            "Stool Calprotectin UCSD": "Gut Inflammation",
                            "Stool Calprotectin YFH": "Gut Inflammation",
                            "Stool Lysozyme": "Gut Inflammation",
                            "Stool Lactoferrin": "Gut Inflammation",
                            "Stool SIgA": "Gut Inflammation",

                             # Short Chain Fatty Acids
                            "% SCFA Acetate": "Short Chain Fatty Acids",
                            "% SCFA Propionate": "Short Chain Fatty Acids",
                            "% SCFA Valerate": "Short Chain Fatty Acids",
                            "% SCFA Butyrate": "Short Chain Fatty Acids",
                            "Total SCFA": "Short Chain Fatty Acids",
                            "Butyrate": "Short Chain Fatty Acids"

}

# Mapping for units to biomarkers
dict_unitsToBiomarkers = {
                            # Vitals
                            "Weight": "lbs",

                            # LIPID Panel
                            "237": "mg/dL",
                            'HDL (High Density Lipoprotein) "good"': "mg/dL",
                            "VLDL Cholesterol": "mg/dL",
                            "IDL Chloresterol": None,
                            "Sum Total Cholesterol": "mg/dL",
                            "Triglycerides": "mg/dL",
                            "UCSD LDL Particle # by NMR": "nmol/L",
                            "UCSD HDL Particle #": "μmol/L",
                            "UCSD Large VLDL Particle #": "nmol/L",
                            "UCSD Small LDL Particle #": "nmol/L",
                            "UCSD Large HDL Particle #": "μmol/L",
                            "UCSD VLDL Particle Size": "nm",
                            "UCSD LDL Particle Size": "nm",
                            "UCSD HDL Particle Size": "nm",
                            "Real-LDL Size Pattern": None,
                            "TG/HDL": None,
                            "Chol/HDL": None,
                            "Lp(a) Chloresterol": None,
                            "APO A1": "mg/dL",
                            "APOLIPOPROTEIN-(B100)": "mg/dL",
                            "Ratio APO B100/AI": None,

                            # Comprehensive Metabolic Panel – Electrolytes
                            "Sodium": "mmol/L",
                            "Potassium": "mmol/L",
                            "Chloride": "mmol/L",
                            "Calcium": "mg/dL",
                            "Phosporus": "mg/dL",
                            "Magnesium": "mg/dL",
                            "Zinc": "µg/dL",
                            "Tot CO2": None,
                            "Bicarbonate": "mmol/L",
                            "Anion Gap": "mmol/L",

                            # Comprehensive Metabolic Panel – remainder
                            "Total Protein": "g/dL",
                            "Albumin": "g/dL",
                            "Globulin": None,
                            "A/G ratio": None,
                            "GGTP": "U/L",
                            "LDH": "U/L",
                            "ALT": "U/L",
                            "AST": "U/L",
                            "AST/ALT ratio": None,
                            "Alkaline phosphatase": "U/L",
                            "Total Bilirubin": "mg/dL",
                            "Direct Bilirubin": None,

                            # Kidney
                            "Blood Urea Nitrogen (BUN)": "mg/dL",
                            "Creatinine": "mg/dL",
                            "BUN/Creatinine Ratio": None,
                            "Uric Acid": None,

                            # Glucose/Insulin Panel
                            "Fasting Glucose": "mg/dL",
                            "Insulin": "µIU/mL",
                            "Hemoglobin A1C": "%",
                            "HOMA-IR (insulin resistance)": None,

                            # Hormone Panel
                            "Free Testosterone": "pg/mL",
                            "Testosterone % Free": "%",
                            "Total Testosterone": "ng/mL",
                            "Sex Hormone Binding Globulin (SHBG)": "nmol/L",
                            "Estradiol": "pg/mL",
                            "DHEA-blood": "µg/dL",
                            "Cortisol Random": None,
                            "TSH": "µIU/mL",
                            "Total T3": "ng/mL",
                            "Thyroxine (T4)": "µg/dL",

                            # Cancer Panel
                            "PSA Screen": None,
                            "CA-19-9 (Pancreatic Cancer)": "U/mL",
                            "CEA": "ng/mL",

                            # Iron Panel
                            "Iron": "µg/dL",
                            "Ferritin": "ng/mL",
                            "Fibrinogen": None,
                            "TIBC": "µg/dL",
                            "UIBC": "µg/dL",
                            "Iron saturation": "%",

                            # Complete Blood Count (CBC) Panel – Red Blood Cell Details
                            "RBC-": "10**6/mL",
                            "HGB-": "g/dL",
                            "HCT": "%",
                            "PLT-": "10**3/mL",
                            "MCV-": "fL",
                            "MCH": "pg/cell",
                            "MCHC-": "g/dL",
                            "RDW": "%",
                            "MPV": "fL",

                            # Complete Blood Count (CBC) Panel  – White Blood Cell Details
                            "WBC-": "10**3/mL",
                            "NEU %": "%",
                            "LYM %": "%",
                            "EOS %": "%",
                            "MONO %": "%",
                            "BASO %": "%",
                            "Neutrophil Count": "10**3/mL",
                            "Lymphocyte Count": "10**3/mL",
                            "Monocyte Count": "10**3/mL",
                            "Esoinophil Count": "10**3/mL",
                            "Basophil Count": "10**3/mL",

                            # Inflammation Panel
                            "High Sensitivity CRP (mg/L)": "mg/L",
                            "Lp-PLA2": None,
                            "IgG": None,
                            "IgA": None,
                            "IgM": None,

                            # Vitamins, Antioxidants, Amino Acid Panel
                            "Homocysteine": "µmol/L",
                            "CoQ10": None,
                            "Selenium": None,
                            "Vitamin E Alpha-Tocopherol": "mg/L",
                            "Vitamin E Gamma-Tocopherol": "mg/L",
                            "Alpha Lipoic Acid": None,
                            "Vitamin D, 25-Hydroxy": "ng/mL",
                            "Vitamin B12": "pg/mL",
                            "Niacin (Vit B3)": None,

                            # Stool Tests
                            "pH": None,

                            # Cultured Bacteria Growth - Expected/Beneficial flora
                            "Bifidobacerium family": None,
                            "E. coli spp.": None,
                            "Lactobacillus spp": None,
                            "Enterococcus spp.": None,
                            "Bacteroides fraglis group": None,
                            "Clostridium spp": None,
                            "Bacteroides family": None,

                            # Cultured Bacteria Growth - Commensal (Imbalanced) flora 
                            "Alpha hemolytic strep": None,
                            "Hemolytic Escherichia coli": None,
                            "Pseudomonas chlororaphis group": None,
                            "Gamma Hemolytic Streptococcus": None,
                            "Hafnia alvei": None,
                            "Mucoid Escherichia coli": None,
                            "Beta strep, group B, strep. Agalactiae": None,
                            "Klebsiella oxytoca": None,
                            "Bacillus spp, pumilus": None,
                            "Comamonas testosteroni": None,
                            "Streptococcus anginosus": None,
                            "Cellulosimicrobium cellulans/funkeil/marinum": None,
                            "Lactococcus lactis": None,
                            "Staphylococcus aureus": None,
                            "Streptococcus salivarius": None,
                            "Streptococcus parasanguinis": None,
                            "Pseudomonas mendocina ": None,
                            "Actinomyces turicensis": None,
                            "Lysinibacillus spp": None,
                            "Streptococcus mutans": None,
                            "Streptococcus australis": None,
                            "Enterobacter asburiae": None,
                            "Pseudomonas stutzeri": None,
                            "Kocuria spp, salsicia, rhizophila, kristinae": None,
                            "Rothia dentocariosa/mucliaginosa": None,
                            "Pantoea spp": None,
                            "Pseudomonas citronellolis": None,
                            "Streptococcus mitis/oralis/cristatus": None,
                            "Staphylococcus haemolyticus": None,
                            "Serratia marcescens": None,
                            "Staphylococcus lugdunensis": None,
                            "Staphylococcus epidermidis": None,
                            "Actinomyces neuii": None,
                            "Streptomyces": None,
                            "Bacillus licheniformis": None,
                            "Corynebacterium falsenii": None,

                            # Dysbiotic bacteria — no units
                            "Morganellla morganii": None,
                            "Providencia stuartii": None,
                            "Klebsiella pneumonia": None,
                            "Providencia rettgeri": None,
                            "Enterobacter cloacae": None,
                            "Pseudomonas aeruginosa": None,
                            "Bacillus cereus": None,
                            "Citrobacter freundii complex": None,
                            "Arcobacter butzieri": None,

                            # Yeast — no units
                            "Candida parapsilosis": None,
                            "Rhodotorula glutinis/mucilaginosa": None,
                            "Candida krusei/inconspicua/lambica": None,
                            "Saccharomyces cerevisiae/boulardi": None,
                            "Geotrichum spp.": None,
                            "Candida pararugosa": None,
                            "Candida lusitaniae": None,
                            "Candida zeylanoides": None,
                            "Trichosporon asahii": None,
                            "Trichosporon ovoides": None,
                            "Trichosporon montevideense": None,
                            "Exophiala dermatitidis": None,
                            "Cryptococcus spp, not neoformans": None,
                            "Cryptococcus diffuens": None,
                            "Candida orthopsilosis": None,

                            # Gut Inflammation
                            "Stool Calprotectin UCSD": "µg/g",
                            "Stool Calprotectin YFH": None,
                            "Stool Lysozyme": None,
                            "Stool Lactoferrin": None,
                            "Stool SIgA": None,

                            # Short Chain Fatty Acids
                            "% SCFA Acetate": "%",
                            "% SCFA Propionate": "%",
                            "% SCFA Valerate": "%",
                            "% SCFA Butyrate": "%",
                            "Total SCFA": "mg/mL",
                            "Butyrate": "mg/mL"

}

# Mapping for ranges to biomarkers
dict_biomarkersToRanges = {
                                # Vitals
                                "Weight": None,

                                # LIPID Panel
                                "237": "<129",
                                'HDL (High Density Lipoprotein) "good"': "40 to 59",
                                "VLDL Cholesterol": "<30",
                                "IDL Chloresterol": "<20",
                                "Sum Total Cholesterol": "<199",
                                "Triglycerides": "30 to 159",
                                "UCSD LDL Particle # by NMR": "<1135",
                                "UCSD HDL Particle #": ">33",
                                "UCSD Large VLDL Particle #": "<2.7",
                                "UCSD Small LDL Particle #": "<634",
                                "UCSD Large HDL Particle #": ">4.2",
                                "UCSD VLDL Particle Size": "<46.7",
                                "UCSD LDL Particle Size": ">20.7",
                                "UCSD HDL Particle Size": ">8.9",
                                "Real-LDL Size Pattern": "A",
                                "TG/HDL": "<2",
                                "Chol/HDL": "<4.5",
                                "Lp(a) Chloresterol": "<10",
                                "APO A1": "94 to 178",
                                "APOLIPOPROTEIN-(B100)": "55 to 140",
                                "Ratio APO B100/AI": None,

                                # Comprehensive Metabolic Panel – Electrolytes
                                "Sodium": "136 to 145",
                                "Potassium": "3.5 to 5.1",
                                "Chloride": "98 to 107",
                                "Calcium": "8.5 to 10.6",
                                "Phosporus": "2.7 to 4.5",
                                "Magnesium": "1.6 to 2.4",
                                "Zinc": "56-134",
                                "Tot CO2": "23-29",
                                "Bicarbonate": "22 to 29",
                                "Anion Gap": "7 to 15",

                                # Comprehensive Metabolic Panel – remainder
                                "Total Protein": "6 to 8",
                                "Albumin": "3.5 to 5.2",
                                "Globulin": "2.0-4.5",
                                "A/G ratio": "1.1-2.3",
                                "GGTP": "<60",
                                "LDH": "135 to 225",
                                "ALT": "0 to 41",
                                "AST": "0-40",
                                "AST/ALT ratio": None,
                                "Alkaline phosphatase": "40 to 120",
                                "Total Bilirubin": "<1.2",
                                "Direct Bilirubin": "<0.2",

                                # Kidney
                                "Blood Urea Nitrogen (BUN)": "8 to 23",
                                "Creatinine": "0.67 to 1.17",
                                "BUN/Creatinine Ratio": "10 to 20",
                                "Uric Acid": "4.0-8.5",

                                # Glucose/Insulin Panel
                                "Fasting Glucose": "70 to 99",
                                "Insulin": "3 to 19",
                                "Hemoglobin A1C": "4.8 to 5.8",
                                "HOMA-IR (insulin resistance)": "<2.8",

                                # Hormone Panel
                                "Free Testosterone": "47-244",
                                "Testosterone % Free": None,
                                "Total Testosterone": "2.8-8",
                                "Sex Hormone Binding Globulin (SHBG)": "15-100",
                                "Estradiol": "<43",
                                "DHEA-blood": "34 to 249",
                                "Cortisol Random": "6 to 23",
                                "TSH": "0.27 to 4.2",
                                "Total T3": "0.8 to 2.0",
                                "Thyroxine (T4)": "4.5 to 10.9",

                                # Cancer Panel
                                "PSA Screen": "<3.99",
                                "CA-19-9 (Pancreatic Cancer)": "30 to 42",
                                "CEA": "<3.8",

                                # Iron Panel
                                "Iron": "59 to 158",
                                "Ferritin": "30 to 400",
                                "Fibrinogen": "<460",
                                "TIBC": "148 to 506",
                                "UIBC": "112 to 346",
                                "Iron saturation": "?",

                                # Complete Blood Count (CBC) Panel – Red Blood Cell Details
                                "RBC-": "4.6 to 6.1",
                                "HGB-": "13.7 to 17.5",
                                "HCT": "40 to 50",
                                "PLT-": "140 to 370",
                                "MCV-": "79 to 95",
                                "MCH": "26 to 32",
                                "MCHC-": "32 to 36",
                                "RDW": "12 to 14",
                                "MPV": "9.4-12.4",

                                # Complete Blood Count (CBC) Panel  – White Blood Cell Details
                                "WBC-": "4 to 10",
                                "NEU %": "41 to 80",
                                "LYM %": "19 to 53",
                                "EOS %": "1 to 7",
                                "MONO %": "5 to 12",
                                "BASO %": "0 to 2",
                                "Neutrophil Count": "1.6 to 7",
                                "Lymphocyte Count": "0.8 to 3.1",
                                "Monocyte Count": "0.2 to 0.8",
                                "Esoinophil Count": "0.1 to 0.5",
                                "Basophil Count": "<0.1",

                                # Inflammation Panel
                                "High Sensitivity CRP (mg/L)": "<4.9",
                                "Lp-PLA2": "<200",
                                "IgG": "700-1600",
                                "IgA": "70-400",
                                "IgM": "40-230",

                                # Vitamins, Antioxidants, Amino Acid Panel
                                "Homocysteine": "0 to 14",
                                "CoQ10": "0.44-1.64",
                                "Selenium": None,
                                "Vitamin E Alpha-Tocopherol": "5.5-18",
                                "Vitamin E Gamma-Tocopherol": "0-6",
                                "Alpha Lipoic Acid": None,
                                "Vitamin D, 25-Hydroxy": "30-80",
                                "Vitamin B12": "232 to 1245",
                                "Niacin (Vit B3)": None,

                                # Stool Tests
                                "pH": "5.8 to 7",

                                # Cultured Bacteria Growth - Expected/Beneficial flora
                                "Bifidobacerium family": None,
                                "E. coli spp.": None,
                                "Lactobacillus spp": None,
                                "Enterococcus spp.": None,
                                "Bacteroides fraglis group": None,
                                "Clostridium spp": None,
                                "Bacteroides family": None,

                                # Cultured Bacteria Growth - Commensal (Imbalanced) flora 
                                "Alpha hemolytic strep": None,
                                "Hemolytic Escherichia coli": None,
                                "Pseudomonas chlororaphis group": None,
                                "Gamma Hemolytic Streptococcus": None,
                                "Hafnia alvei": None,
                                "Mucoid Escherichia coli": None,
                                "Beta strep, group B, strep. Agalactiae": None,
                                "Klebsiella oxytoca": None,
                                "Bacillus spp, pumilus": None,
                                "Comamonas testosteroni": None,
                                "Streptococcus anginosus": None,
                                "Cellulosimicrobium cellulans/funkeil/marinum": None,
                                "Lactococcus lactis": None,
                                "Staphylococcus aureus": None,
                                "Streptococcus salivarius": None,
                                "Streptococcus parasanguinis": None,
                                "Pseudomonas mendocina ": None,
                                "Actinomyces turicensis": None,
                                "Lysinibacillus spp": None,
                                "Streptococcus mutans": None,
                                "Streptococcus australis": None,
                                "Enterobacter asburiae": None,
                                "Pseudomonas stutzeri": None,
                                "Kocuria spp, salsicia, rhizophila, kristinae": None,
                                "Rothia dentocariosa/mucliaginosa": None,
                                "Pantoea spp": None,
                                "Pseudomonas citronellolis": None,
                                "Streptococcus mitis/oralis/cristatus": None,
                                "Staphylococcus haemolyticus": None,
                                "Serratia marcescens": None,
                                "Staphylococcus lugdunensis": None,
                                "Staphylococcus epidermidis": None,
                                "Actinomyces neuii": None,
                                "Streptomyces": None,
                                "Bacillus licheniformis": None,
                                "Corynebacterium falsenii": None,

                                # Dysbiotic bacteria — no ranges
                                "Morganellla morganii": None,
                                "Providencia stuartii": None,
                                "Klebsiella pneumonia": None,
                                "Providencia rettgeri": None,
                                "Enterobacter cloacae": None,
                                "Pseudomonas aeruginosa": None,
                                "Bacillus cereus": None,
                                "Citrobacter freundii complex": None,
                                "Arcobacter butzieri": None,

                                # Yeast — no ranges
                                "Candida parapsilosis": None,
                                "Rhodotorula glutinis/mucilaginosa": None,
                                "Candida krusei/inconspicua/lambica": None,
                                "Saccharomyces cerevisiae/boulardi": None,
                                "Geotrichum spp.": None,
                                "Candida pararugosa": None,
                                "Candida lusitaniae": None,
                                "Candida zeylanoides": None,
                                "Trichosporon asahii": None,
                                "Trichosporon ovoides": None,
                                "Trichosporon montevideense": None,
                                "Exophiala dermatitidis": None,
                                "Cryptococcus spp, not neoformans": None,
                                "Cryptococcus diffuens": None,
                                "Candida orthopsilosis": None,

                                # Gut Inflammation
                                "Stool Calprotectin UCSD": "<50",
                                "Stool Calprotectin YFH": "<80",
                                "Stool Lysozyme": "<500",
                                "Stool Lactoferrin": "<7.3",
                                "Stool SIgA": "30-275",

                                # Short Chain Fatty Acids
                                "% SCFA Acetate": "50 to 72",
                                "% SCFA Propionate": "11 to 25",
                                "% SCFA Valerate": "1 to 5",
                                "% SCFA Butyrate": "11 to 32",
                                "Total SCFA": "5 to 16",
                                "Butyrate": "0.8-4"
}

In [71]:
# Drop Columns with Low, High, Average,	Number	
df_longBiomarkers= df_biomarkers.drop(columns=['Low', 'High', 'Average', 'Number'])

# Collect columns
cols = list(df_longBiomarkers.columns)

# Create columns for biomarker, dates, facility



In [72]:
col_biometricPanel = find_exact("Date collected") or cols[0]  # biomarker/test label
col_units = find_exact("Units")
col_range = find_exact("Good Range")
cols_meta = {c for c in [col_biometricPanel, col_units, col_range] if c is not None}

cols_drop = {"low", "high", "average", "number"}

# Collect date column
rgx_date = re.compile(r"\s*\d{1,2}/\d{1,2}/\d{2,4}\s*")  # must look like M/D/YYYY
col_date = [
                c for c in df_biomarkers.columns
                if (c not in cols_meta)
                and (str(c).strip().lower() not in cols_drop)
                and rgx_date.fullmatch(str(c))  # must look like M/D/YYYY
]

col_date = [c for c in col_date if not str(c).lower().startswith("unnamed")]

# Facility column
row_one = df_biomarkers.iloc[0]
dict_facilityByDate = {
                        dc: (None if is_blank(row_one.get(dc)) 
                        else str(row_one.get(dc)).strip())
                        for dc in col_date
}


In [73]:
# Collect Biomarkers and Panel Columns from the Date Collected column

lst_panels = [
    "Vitals",
    "LIPID Panel",
    "Comprehensive Metabolic Panel",
    "Kidney",
    "Glucose/Insulin Panel",
    "Hormone Panel",
    "Cancer Panel",
    "Iron Panel",
    "CBC: Red Blood Cell Details",
    "CBC: White Blood Cell Details",
    "Inflammation Panel",
    "Vitamins, Antioxidants, Amino Acid Panel",
    "Stool tests",
    "Expected/Beneficial flora",
    "Commensal (Imbalanced) flora",
    "Dysbiotic bacteria",
    "Yeast",
    "Gut Inflammation",
    "Short Chain Fatty Acids",
]

# --- pull biomarkers from the "Date collected" column and make clean lists ---

# 0) left column cleaned
labels = [clean_label(x) for x in df_biomarkers[col_biometricPanel].tolist()]

# 1) drop blanks/separators
labels_noblank = [x for x in labels if x is not None]

# 2) panels found (in-order, unique) limited to your fixed lst_panels
lst_panels_found = uniq_order([x for x in labels_noblank if x in lst_panels])

labels_noblank, lst_panels_found 


(['Weight',
  'Blood Tests',
  'LIPID Panel',
  '237',
  'HDL (High Density Lipoprotein) "good"',
  'VLDL Cholesterol',
  'IDL Chloresterol',
  'Sum Total Cholesterol',
  'Triglycerides',
  'UCSD LDL Particle # by NMR',
  'UCSD HDL Particle #',
  'UCSD Large VLDL Particle #',
  'UCSD Small LDL Particle #',
  'UCSD Large HDL Particle #',
  'UCSD VLDL Particle Size',
  'UCSD LDL Particle Size',
  'UCSD HDL Particle Size',
  'Real-LDL Size Pattern',
  'TG/HDL',
  'Chol/HDL',
  'Lp(a) Chloresterol',
  'APO A1',
  'APOLIPOPROTEIN-(B100)',
  'Ratio APO B100/AI',
  'Comprehensive Metabolic Panel',
  'Electolytes',
  'Sodium',
  'Potassium',
  'Chloride',
  'Calcium',
  'Phosporus',
  'Magnesium',
  'Zinc',
  'Tot CO2',
  'Bicarbonate',
  'Anion Gap',
  'Liver',
  'Total Protein',
  'Albumin',
  'Globulin',
  'A/G ratio',
  'GGTP',
  'LDH',
  'ALT',
  'AST',
  'AST/ALT ratio',
  'Alkaline phosphatase',
  'Total Bilirubin',
  'Direct Bilirubin',
  'Kidney',
  'Blood Urea Nitrogen (BUN)',
  'Cre

In [74]:
# Extract Biomarkrker value which are the column/row, date, biomarker

# rows that are biomarkers (not panel headers)
idx_biomarker_rows = [
                        i for i, x in enumerate(labels)
                        if (x is not None) and (x in dict_biomarkerToPanels)
]

# build long rows directly (no melt, no wide step)
records = []
for i in idx_biomarker_rows:
    biomarker = labels[i]  # already cleaned
    row = df_biomarkers.loc[i, col_date]
    for dc, val in row.items():
        if not is_blank(val):
            records.append({"Date": dc, "Biomarker": biomarker, "Value": val})

df_longBiomarkers = pd.DataFrame.from_records(records)

# tidy types and order (still long)
df_longBiomarkers["Date"]  = pd.to_datetime(df_longBiomarkers["Date"], errors="coerce")
df_longBiomarkers["Value"] = pd.to_numeric(df_longBiomarkers["Value"], errors="ignore")
df_longBiomarkers = df_longBiomarkers.sort_values(["Date","Biomarker"]).reset_index(drop=True)
df_longBiomarkers

/var/folders/nf/dp5pbtr55rn8mww76lsrkt800000gn/T/ipykernel_80014/766801454.py:22: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_longBiomarkers["Value"] = pd.to_numeric(df_longBiomarkers["Value"], errors="ignore")


,Date,Biomarker,Value
0,1993-05-01,Sum Total Cholesterol,152.0
1,1996-11-27,A/G ratio,1.3
2,1996-11-27,AST,37.0
3,1996-11-27,Albumin,4.2
4,1996-11-27,Alkaline phosphatase,64.0
...,...,...,...
22857,2025-06-13,"Vitamin D, 25-Hydroxy",69
22858,2025-06-13,Vitamin E Alpha-Tocopherol,7.9
22859,2025-06-13,Vitamin E Gamma-Tocopherol,0.6
22860,2025-06-13,WBC-,4.4


In [75]:
# make units, panel, ranges, and facilitie columns by mapping

df_longBiomarkers["Units"] = df_longBiomarkers["Biomarker"].map(dict_unitsToBiomarkers)
df_longBiomarkers["Panel"] = df_longBiomarkers["Biomarker"].map(dict_biomarkerToPanels)
df_longBiomarkers["GoodRange"] = df_longBiomarkers["Biomarker"].map(dict_biomarkersToRanges)
df_longBiomarkers["Facility"] = df_longBiomarkers["Date"].map(dict_facilityByDate)
df_longBiomarkers

,Date,Biomarker,Value,Units,Panel,GoodRange,Facility
0,1993-05-01,Sum Total Cholesterol,152.0,mg/dL,LIPID Panel,<199,NaN
1,1996-11-27,A/G ratio,1.3,None,Comprehensive Metabolic Panel,1.1-2.3,NaN
2,1996-11-27,AST,37.0,U/L,Comprehensive Metabolic Panel,0-40,NaN
3,1996-11-27,Albumin,4.2,g/dL,Comprehensive Metabolic Panel,3.5 to 5.2,NaN
4,1996-11-27,Alkaline phosphatase,64.0,U/L,Comprehensive Metabolic Panel,40 to 120,NaN
...,...,...,...,...,...,...,...
22857,2025-06-13,"Vitamin D, 25-Hydroxy",69,ng/mL,"Vitamins, Antioxidants, Amino Acid Panel",30-80,NaN
22858,2025-06-13,Vitamin E Alpha-Tocopherol,7.9,mg/L,"Vitamins, Antioxidants, Amino Acid Panel",5.5-18,NaN
22859,2025-06-13,Vitamin E Gamma-Tocopherol,0.6,mg/L,"Vitamins, Antioxidants, Amino Acid Panel",0-6,NaN
22860,2025-06-13,WBC-,4.4,10**3/mL,CBC: White Blood Cell Details,4 to 10,NaN


In [76]:
# 1) rebuild facility dict with normalized datetime keys; skip bad/blank keys
_fixed = {}
for k, v in dict_facilityByDate.items():
    if is_blank(k):
        continue
    ts = pd.to_datetime(str(k).strip(), errors="coerce")
    if pd.isna(ts):
        continue
    _fixed[ts.normalize()] = (None if is_blank(v) else str(v).strip())
dict_facilityByDate = _fixed  # overwrite with safe version

# 2) map facility WITHOUT changing df_longBiomarkers["Date"]
df_biomarkers["Facility"] = pd.to_datetime(
    df_longBiomarkers["Date"], errors="coerce"
).dt.normalize().map(dict_facilityByDate)

# quick check
print(df_longBiomarkers["Facility"].value_counts(dropna=False).head())

Facility
NaN    22862
Name: count, dtype: int64


In [77]:
df_longBiomarkers["Facility"].value_counts()

Series([], Name: count, dtype: int64)

In [78]:
df_longBiomarkers.to_csv(fp_biomarkersLong,index=False)
print(f"Saved {fp_biomarkersLong}")

Saved Data/LLM_Biomarkers_Long.csv


## Make Wide Format

In [79]:
# Creat wide dataframe from longBioDairyLS

# Change column labels to biomarker_unit excpet if "%" contained in biomarker
df_longBiomarkers["Biomarker_Unit"] = df_longBiomarkers.apply(
                                                                lambda x: f"{x['Biomarker']}_{x['Units']}"
                                                                if "%" not in x["Biomarker"] else x["Biomarker"],
                                                                axis=1
)

# Pivot to make date the rows and biomarkers the columns
df_wideBiomarkers = df_longBiomarkers.pivot(
                                            index="Date", 
                                            columns="Biomarker_Unit", 
                                            values="Value"
).reset_index()
df_wideBiomarkers

Biomarker_Unit,Date,% SCFA Acetate,% SCFA Butyrate,% SCFA Propionate,% SCFA Valerate,237_mg/dL,A/G ratio_None,ALT_U/L,APO A1_mg/dL,APOLIPOPROTEIN-(B100)_mg/dL,...,Uric Acid_None,VLDL Cholesterol_mg/dL,Vitamin B12_pg/mL,"Vitamin D, 25-Hydroxy_ng/mL",Vitamin E Alpha-Tocopherol_mg/L,Vitamin E Gamma-Tocopherol_mg/L,WBC-_10**3/mL,Weight_lbs,Zinc_µg/dL,pH_None
0,1993-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-11-27,NaN,NaN,NaN,NaN,NaN,1.3,NaN,NaN,NaN,...,6.0,NaN,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN
2,1997-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.3,NaN,NaN,NaN,NaN,NaN,6.3,NaN,NaN,NaN
3,1998-04-15,NaN,NaN,NaN,NaN,NaN,1.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.4,NaN,NaN,NaN
4,1999-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,2025-02-05,47,24,22,7.2,NaN,NaN,22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.7,178,NaN,6.5
328,2025-02-12,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN,...,NaN,NaN,NaN,49,NaN,NaN,4,179,NaN,NaN
329,2025-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.5,179.6,NaN,NaN
330,2025-05-20,NaN,NaN,NaN,NaN,NaN,NaN,17,NaN,NaN,...,NaN,NaN,NaN,64,NaN,NaN,4,182.5,NaN,NaN


In [80]:
df_wideBiomarkers.to_csv(fp_biomarkersWide, index=False)
print(f"Saved {fp_biomarkersWide}")

Saved Data/LLM_Biomarkers_Wide.csv


# Gut Microbiome Data

In [81]:
df_microbiome = pd.read_csv(fp_microbiome)
df_microbiome

,LS Gut Microbiome,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239
0,Sequenced in Knight Lab from LS Frozen Stool S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date stool sample collected,NaN,Max,Average,12/28/11,4/3/12,8/7/12,11/6/12,1/26/13,2/8/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16S Relative Abundance,NaN,NaN,NaN,16S,16S,16S,16S,16S,16S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sum of 16S Relative Abundances,NaN,NaN,NaN,97.15,97.89,97.98,98.39,97.89,98.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Dominate species from Metagenomics comparison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Escherichia coli Max HE,NaN,NaN,NaN,0.6032,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,Morganella morganii Ave HE,NaN,NaN,NaN,0.000108823529411765,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,Morganella morganii Max HE,NaN,NaN,NaN,0.0004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,Bifidobacterium longum Ave HE,NaN,NaN,NaN,0.108320588235294,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create Summary Dataframe

### Helper Functions

In [82]:
def get_row_values(wanted_label: str):
    target = clean_label(wanted_label)
    idx = labels_clean[labels_clean == target].index
    if len(idx) == 0:
        return pd.Series(index=col_date, dtype=float)
    return df_microbiome.loc[idx[0], col_date]

In [83]:
def get_row_values(wanted_label: str):
    """
    Return a Series indexed by your existing `col_date` labels, pulling values
    from the row whose left label == wanted_label. Works even if the displayed
    dates live in a header row and the actual DataFrame column names differ.
    """
    target = clean_label(wanted_label)
    idx = labels_clean[labels_clean == target].index
    if len(idx) == 0:
        return pd.Series(index=col_date, dtype=float)

    # use your already-detected header row `i` if present; otherwise find it
    try:
        header_row_idx = i
    except NameError:
        _hdr = labels_clean[labels_clean == "Date stool sample collected"].index
        header_row_idx = _hdr[0] if len(_hdr) else idx[0]

    # map displayed date labels -> physical DataFrame column names
    date_set = set(col_date)
    date_to_col = {}
    for c in cols:
        v = clean_label(df_microbiome.at[header_row_idx, c])
        if v in date_set and v not in date_to_col:
            date_to_col[v] = c

    r = idx[0]
    # build a Series indexed by your `col_date` (return NaN where unmapped)
    return pd.Series(
        [df_microbiome.at[r, date_to_col[d]] if d in date_to_col else None for d in col_date],
        index=col_date
    )

In [84]:
df_microbiome = pd.read_csv(fp_microbiome)
df_microbiome

,LS Gut Microbiome,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239
0,Sequenced in Knight Lab from LS Frozen Stool S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Date stool sample collected,NaN,Max,Average,12/28/11,4/3/12,8/7/12,11/6/12,1/26/13,2/8/13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16S Relative Abundance,NaN,NaN,NaN,16S,16S,16S,16S,16S,16S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sum of 16S Relative Abundances,NaN,NaN,NaN,97.15,97.89,97.98,98.39,97.89,98.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Dominate species from Metagenomics comparison,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Escherichia coli Max HE,NaN,NaN,NaN,0.6032,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
282,Morganella morganii Ave HE,NaN,NaN,NaN,0.000108823529411765,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,Morganella morganii Max HE,NaN,NaN,NaN,0.0004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,Bifidobacterium longum Ave HE,NaN,NaN,NaN,0.108320588235294,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
cols = list(df_microbiome.columns) # all columns
col_label = find_exact("Date stool sample collected") or cols[0] # left label column (use your finder)

# locate the row whose label is exactly "Date stool sample collected"
labels_clean = df_microbiome[col_label].map(clean_label)
i = labels_clean[labels_clean == "Date stool sample collected"].index[0]

# grab the whole row, then drop the non-date entries
row_vals = [clean_label(v) for v in df_microbiome.loc[i, cols].tolist()]
col_date = [    
            v for v in row_vals
            if v not in (None, "Date stool sample collected", "Max", "Average")
]

# ensure uniqueness while preserving the original order
col_date = uniq_order(col_date)

# pull exactly the three rows you asked for
sum16s   = pd.to_numeric(get_row_values("Sum of 16S Relative Abundances"), errors="coerce")
sum_meta = pd.to_numeric(get_row_values("Sum of Metagenomics Relative Abundances"), errors="coerce")
shannon  = pd.to_numeric(get_row_values("Shannon Diversity Index Score Computed from 16S"), errors="coerce")

# assemble the summary strictly from your date_cols (no extra logic)
df_microbiomeSummary = (
                          pd.DataFrame(index=col_date)
                            .assign(Sum16S=sum16s)
                            .assign(SumMetagenomics=sum_meta)
                            .assign(Shannon16S=shannon)
                            .rename_axis("Date")
                            .reset_index()
                            .dropna(how="all", subset=["Sum16S", "SumMetagenomics", "Shannon16S"])
)

df_microbiomeSummary


,Date,Sum16S,SumMetagenomics,Shannon16S
0,12/28/11,97.15,99.957,5.444
1,4/3/12,97.89,99.985,4.633
2,8/7/12,97.98,99.965,5.997
3,11/6/12,98.39,99.983,5.244
4,1/26/13,97.89,99.987,6.235
...,...,...,...,...
154,6/16/19,99.06,NaN,6.082
155,6/23/19,99.10,NaN,5.596
156,7/14/19,99.00,NaN,6.007
157,7/28/19,98.52,NaN,6.133


In [86]:
# Write out the summary
df_microbiomeSummary.to_csv(fp_microbiomeSummary, index=False)

## Create Long Format

### Mapping Dictionaries

In [87]:
# Species to Panel mapping
dict_speciesToPanels = {

                            # 16S
                            "p__Proteobacteria;cf__Enterobacteriaceae;g__Morganell": "16S",  
                            "p__Proteobacteria;cf__Enterobacteriaceae;g__Morganella": "16S",
                            "p__Proteobacteria;f__Enterobacteriaceae;__": "16S",
                            "p__Proteobacteria;f__Enterobacteriaceae;g__": "16S",
                            "p__Bacteroidetes;f__Bacteroidaceae;g__Bacteroides": "16S",
                            "p__Firmicutes;f__Ruminococcaceae;g__": "16S",
                            "p__Verrucomicrobia;f__Verrucomicrobiaceae;g__Akkermansia": "16S",
                            "p__Firmicutes;f__;g__": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Coprococcus": "16S",
                            "p__Bacteroidetes;f__Prevotellaceae;g__Prevotella": "16S",
                            "p__Bacteroidetes;f__Rikenellaceae;g__": "16S",
                            "p__Firmicutes;f__Ruminococcaceae;g__Faecalibacterium": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Lachnospira": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Blautia": "16S",
                            "p__Firmicutes;f__Christensenellaceae;g__": "16S",
                            "p__Euryarchaeota;f__Methanobacteriaceae;g__Methanobrevibacter": "16S",
                            "p__Bacteroidetes;f__Porphyromonadaceae;g__Parabacteroides": "16S",
                            "p__Proteobacteria;f__Enterobacteriaceae;g__Providencia": "16S",
                            "p__Firmicutes;f__Ruminococcaceae;g__Ruminococcus": "16S",
                            "p__Bacteroidetes;f__[Paraprevotellaceae];g__[Prevotella]": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__[Ruminococcus]": "16S",
                            "p__Proteobacteria;f__Enterobacteriaceae;g__Serratia": "16S",
                            "p__Firmicutes;f__Veillonellaceae;g__Dialister": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Dorea": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Roseburia": "16S",
                            "p__Firmicutes;f__Erysipelotrichaceae;g__": "16S",
                            "p__Bacteroidetes;f__S24-7;g__": "16S",
                            "p__Proteobacteria;f__Pseudomonadaceae;g__Pseudomonas": "16S",
                            "p__Proteobacteria;f__;g__": "16S",
                            "p__Fusobacteria;f__Fusobacteriaceae;g__Fusobacterium": "16S",
                            "p__Bacteroidetes;f__[Barnesiellaceae];g__": "16S",
                            "p__Firmicutes;f__Veillonellaceae;g__Phascolarctobacterium": "16S",
                            "p__Proteobacteria;f__Enterobacteriaceae;g__Erwinia": "16S",
                            "p__Proteobacteria;f__Alcaligenaceae;g__Sutterella": "16S",
                            "p__Proteobacteria;f__Desulfovibrionaceae;g__Bilophila": "16S",
                            "p__Cyanobacteria;f__;g__": "16S",
                            "p__Firmicutes;f__Ruminococcaceae;g__Oscillospira": "16S",
                            "p__Firmicutes;f__Erysipelotrichaceae;g__[Eubacterium]": "16S",
                            "p__Proteobacteria;f__Desulfovibrionaceae;g__Desulfovibrio": "16S",
                            "p__Tenericutes;f__;g__": "16S",
                            "p__Proteobacteria;f__Burkholderiaceae;g__": "16S",
                            "p__Synergistetes;f__Synergistaceae;g__Synergistes": "16S",
                            "p__Firmicutes;f__[Tissierellaceae];g__Parvimonas": "16S",
                            "p__Firmicutes;f__Clostridiaceae;g__": "16S",
                            "p__Actinobacteria;f__Coriobacteriaceae;g__": "16S",
                            "p__Firmicutes;f__Clostridiaceae;g__Clostridium": "16S",
                            "p__Firmicutes;f__Veillonellaceae;g__Veillonella": "16S",
                            "p__Proteobacteria;f__Pasteurellaceae;g__Aggregatibacter": "16S",
                            "p__Actinobacteria;f__Bifidobacteriaceae;g__Bifidobacterium": "16S",
                            "p__Firmicutes;f__Lachnospiraceae;g__Lachnobacterium": "16S",

                            # Xin Fang MetaphlAN 2 Metagenomics 
                            "Sum of Metagenomics Relative Abundances": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Escherichia_coli": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Akkermansia_muciniphila": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Faecalibacterium_prausnitzii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_rectale": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Methanobrevibacter_smithii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Subdoligranulum_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_uniformis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_vulgatus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_eligens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_intestinalis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Morganella_morganii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_siraeum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_cellulosilyticus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Dialister_invisus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Escherichia_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Citrobacter_freundii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_sp_L2_50": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_pectinophilus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_caccae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_lactaris": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_putredinis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parabacteroides_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Roseburia_inulinivorans": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Enterobacteria_phage_HK97": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_nexile": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_1_1_57FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_bromii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Klebsiella_pneumoniae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parabacteroides_merdae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Sutterella_wadsworthensis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Barnesiella_intestinihominis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "C2likevirus_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_torques": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroidales_bacterium_ph8": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_thetaiotaomicron": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_ovatus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Roseburia_intestinalis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Roseburia_hominis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Providencia_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Enterobacteriaceae_bacterium_9_2_54FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_salyersiae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_fragilis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Dorea_longicatena": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_sp_HGB5": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_sp_5_1_39BFAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parvimonas_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_shahii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_obeum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_hallii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Coprococcus_sp_ART55_1": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bifidobacterium_longum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_onderdonkii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Fusobacterium_nucleatum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "T5likevirus_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Odoribacter_splanchnicus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_dorei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Coprococcus_comes": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Oscillibacter_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bilophila_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Collinsella_aerofaciens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Klebsiella_oxytoca": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_thermophilus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Dorea_formicigenerans": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_ramulus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_3_1_46FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_3_1_57FAA_CT1": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_sp_AP11": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Subdoligranulum_sp_4_3_54A2FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Klebsiella_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Coprobacillus_sp_29_1": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_5_1_63FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Methanosphaera_stadtmanae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Roseburia_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_hathewayi": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Aggregatibacter_segnis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Enterobacter_cloacae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Anaerostipes_hadrus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Peptostreptococcus_stomatis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Desulfovibrio_piger": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parabacteroides_goldsteinii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_symbiosum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Coprococcus_catus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bifidobacterium_animalis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parvimonas_micra": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Anaerotruncus_colihominis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Porphyromonas_asaccharolytica": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Oxalobacter_formigenes": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Citrobacter_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_bolteae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Veillonella_parvula": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_xylanisolvens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Gemella_morbillorum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Solobacterium_moorei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_indistinctus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_salivarius": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Erysipelotrichaceae_bacterium_2_2_44A": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_parasanguinis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pseudomonas_phage_F116": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_leptum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_8_1_57FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_7_1_58FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Holdemania_filiformis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Veillonella_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bilophila_wadsworthia": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Burkholderia_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lactobacillus_casei_paracasei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Fusobacterium_ulcerans": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Adlercreutzia_equolifaciens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_senegalensis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Aggregatibacter_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Erysipelotrichaceae_bacterium_6_1_45": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pseudomonas_aeruginosa": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Burkholderiales_bacterium_1_1_47": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eggerthella_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Anaerotruncus_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_ventriosum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridiaceae_bacterium_JC118": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcaceae_bacterium_D16": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Anaerococcus_vaginalis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Haemophilus_parainfluenzae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Methanobrevibacter_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eikenella_corrodens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_infirmum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_anginosus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Burkholderia_cenocepacia": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_asparagiforme": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Granulicatella_adiacens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_scindens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Coprobacter_fastidiosus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Alistipes_finegoldii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_nordii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_australis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Ruminococcus_callidus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Hafnia_alvei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bifidobacterium_breve": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parasutterella_excrementihominis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lactococcus_phage_P680": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Veillonella_atypica": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lactococcus_lactis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pseudomonas_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lactobacillus_helveticus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridiales_bacterium_1_7_47FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lactobacillus_acidophilus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Blautia_hydrogenotrophica": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Aeromonas_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pantoea_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_citroniae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Dialister_succinatiphilus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Peptoniphilus_harei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Parabacteroides_distasonis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Gordonibacter_pamelaeae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Fusobacterium_periodonticum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Shigella_sonnei": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Flavonifractor_plautii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eggerthella_lenta": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Marvinbryantia_formatexigens": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Erysipelotrichaceae_bacterium_21_3": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Bacteroides_faecis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_innocuum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Peptoniphilus_lacrimalis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Clostridium_bartlettii": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Campylobacter_showae": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Rothia_mucilaginosa": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Peptostreptococcaceae_noname_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Enterococcus_faecium": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pediococcus_acidilactici": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Desulfovibrio_desulfuricans": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Leuconostoc_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_vestibularis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Lachnospiraceae_bacterium_5_1_57FAA": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Streptococcus_constellatus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Torque_teno_virus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Holdemania_unclassified": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Pseudoflavonifractor_capillosus": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Prevotella_timonensis": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Eubacterium_limosum": "Xin Fang MetaphlAN 2 Metagenomics",
                            "Oscillibacter_sp_KLE_1745": "Xin Fang MetaphlAN 2 Metagenomics"
                                                        
}

# Map Species to Dominant Species for 16S only (some species map to None)
dict_speciesToDominantSpecies16S = {
                                    "p__Proteobacteria;cf__Enterobacteriaceae;g__Morganella": "Morganella_morganii",
                                    "p__Proteobacteria;f__Enterobacteriaceae;__": "Klebsiella_pneumoniae",
                                    "p__Proteobacteria;f__Enterobacteriaceae;g__": "Escherichia_coli",
                                    "p__Bacteroidetes;f__Bacteroidaceae;g__Bacteroides": "Bacteroides_uniformis",
                                    "p__Firmicutes;f__Ruminococcaceae;g__": "Subdoligranulum_unclassified",
                                    "p__Verrucomicrobia;f__Verrucomicrobiaceae;g__Akkermansia": "Akkermansia_muciniphila",
                                    "p__Firmicutes;f__;g__": "Clostridium_sp_L2_50",
                                    "p__Firmicutes;f__Lachnospiraceae;g__Coprococcus": None,
                                    "p__Bacteroidetes;f__Prevotellaceae;g__Prevotella": None,
                                    "p__Bacteroidetes;f__Rikenellaceae;g__": "Alistipes_putredinis",
                                    "p__Firmicutes;f__Ruminococcaceae;g__Faecalibacterium": "Faecalibacterium_prausnitzii",
                                    "p__Firmicutes;f__Lachnospiraceae;g__": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__Lachnospira": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__Blautia": None,
                                    "p__Firmicutes;f__Christensenellaceae;g__": None,
                                    "p__Euryarchaeota;f__Methanobacteriaceae;g__Methanobrevibacter": "Methanobrevibacter_smithii",
                                    "p__Bacteroidetes;f__Porphyromonadaceae;g__Parabacteroides": None,
                                    "p__Proteobacteria;f__Enterobacteriaceae;g__Providencia": None,
                                    "p__Firmicutes;f__Ruminococcaceae;g__Ruminococcus": None,
                                    "p__Bacteroidetes;f__[Paraprevotellaceae];g__[Prevotella]": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__[Ruminococcus]": None,
                                    "p__Proteobacteria;f__Enterobacteriaceae;g__Serratia": None,
                                    "p__Firmicutes;f__Veillonellaceae;g__Dialister": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__Dorea": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__Roseburia": None,
                                    "p__Firmicutes;f__Erysipelotrichaceae;g__": None,
                                    "p__Bacteroidetes;f__S24-7;g__": None,
                                    "p__Proteobacteria;f__Pseudomonadaceae;g__Pseudomonas": None,
                                    "p__Proteobacteria;f__;g__": None,
                                    "p__Fusobacteria;f__Fusobacteriaceae;g__Fusobacterium": None,
                                    "p__Bacteroidetes;f__[Barnesiellaceae];g__": None,
                                    "p__Firmicutes;f__Veillonellaceae;g__Phascolarctobacterium": None,
                                    "p__Proteobacteria;f__Enterobacteriaceae;g__Erwinia": None,
                                    "p__Proteobacteria;f__Alcaligenaceae;g__Sutterella": None,
                                    "p__Proteobacteria;f__Desulfovibrionaceae;g__Bilophila": None,
                                    "p__Cyanobacteria;f__;g__": None,
                                    "p__Firmicutes;f__Ruminococcaceae;g__Oscillospira": None,
                                    "p__Firmicutes;f__Erysipelotrichaceae;g__[Eubacterium]": None,
                                    "p__Proteobacteria;f__Desulfovibrionaceae;g__Desulfovibrio": None,
                                    "p__Tenericutes;f__;g__": None,
                                    "p__Proteobacteria;f__Burkholderiaceae;g__": None,
                                    "p__Synergistetes;f__Synergistaceae;g__Synergistes": None,
                                    "p__Firmicutes;f__[Tissierellaceae];g__Parvimonas": None,
                                    "p__Firmicutes;f__Clostridiaceae;g__": None,
                                    "p__Actinobacteria;f__Coriobacteriaceae;g__": None,
                                    "p__Firmicutes;f__Clostridiaceae;g__Clostridium": None,
                                    "p__Firmicutes;f__Veillonellaceae;g__Veillonella": None,
                                    "p__Proteobacteria;f__Pasteurellaceae;g__Aggregatibacter": None,
                                    "p__Actinobacteria;f__Bifidobacteriaceae;g__Bifidobacterium": None,
                                    "p__Firmicutes;f__Lachnospiraceae;g__Lachnobacterium": None
}

dict_speciesToDominantXF = {
                                    "Xin Fang MetaphlAN 2 Metagenomics" : None
}

# Healthy reference (Average)
dict_speciesToHealthyAverage = {
                                "Bacteroides_vulgatus": 6.05498529411765,
                                "Bacteroides_ovatus": 4.23238823529412,
                                "Prevotella_copri": 3.30236764705882,
                                "Eubacterium_rectale": 3.05285588235294,
                                "Faecalibacterium_prausnitzii": 2.95155,
                                "Dialister_invisus": 2.16144117647059,
                                "Bacteroides_dorei": 2.11637352941176,
                                "Ruminococcus_bromii": 1.85948235294118,
                                "Bacteroides_uniformis": 1.7114,
                                "Bacteroides_cellulosilyticus": 1.48938235294118,
                                "Alistipes_finegoldii": 1.15306764705882,
                                "Akkermansia_muciniphila": 0.984429411764706,
                                "Eubacterium_siraeum": 0.966767647058824,
                                "Eubacterium_eligens": 0.939023529411765,
                                "Bacteroides_fragilis": 0.905426470588236,
                                "Methanobrevibacter_smithii": 0.111332352941176,
                                "Parvimonas_micra": 0.0172323529411765,
                                "Streptococcus_thermophilus": 0.0225911764705882,
                                "Bacteroides_intestinalis": 0.630638235294118,
                                "Escherichia_coli": 0.0551147058823529,
                                "Morganella_morganii": 0.000108823529411765,
                                "Bifidobacterium_longum": 0.108320588235294
}

# Healthy reference (Max)
dict_speciesToHealthyMax = {
                            "Bacteroides_vulgatus": 26.5957,
                            "Bacteroides_ovatus": 41.7481,
                            "Prevotella_copri": 46.0315,
                            "Eubacterium_rectale": 21.8571,
                            "Faecalibacterium_prausnitzii": 11.7871,
                            "Dialister_invisus": 27.0459,
                            "Bacteroides_dorei": 9.7387,
                            "Ruminococcus_bromii": 9.6994,
                            "Bacteroides_uniformis": 4.2107,
                            "Bacteroides_cellulosilyticus": 18.1811,
                            "Alistipes_finegoldii": 10.7227,
                            "Akkermansia_muciniphila": 8.439,
                            "Eubacterium_siraeum": 12.1935,
                            "Eubacterium_eligens": 4.6383,
                            "Bacteroides_fragilis": 4.1907,
                            "Methanobrevibacter_smithii": 1.295,
                            "Parvimonas_micra": 0.0718,
                            "Streptococcus_thermophilus": 0.1446,
                            "Bacteroides_intestinalis": 6.4841,
                            "Escherichia_coli": 0.6032,
                            "Morganella_morganii": 0.0004,
                            "Bifidobacterium_longum": 1.2247
}


In [88]:
# Create long format
# Species, Panel, DominantSpecies, HealthyAverage, HealthyMax, Abundance
lst_species = dict_speciesToPanels.keys()

# Mathch each entry for abundance to speciies and date
for sp in lst_species:
    s = pd.to_numeric(get_row_values(sp), errors="coerce")  # indexed by your existing col_date
    for date in col_date:
        val = s.get(date)
        if pd.notna(val):
            records.append({"Date": date, "Species": sp, "Abundance": val})

df_longMicrobiome = pd.DataFrame.from_records(records)
df_longMicrobiome 



,Date,Biomarker,Value,Species,Abundance
0,10/6/1999,Weight,202.0,NaN,NaN
1,5/7/2001,Weight,201.0,NaN,NaN
2,11/13/2003,Weight,192.0,NaN,NaN
3,10/4/2005,Weight,179.0,NaN,NaN
4,4/17/2007,Weight,179.0,NaN,NaN
...,...,...,...,...,...
38682,7/11/15,NaN,NaN,Oscillibacter_sp_KLE_1745,0.0
38683,7/12/15,NaN,NaN,Oscillibacter_sp_KLE_1745,0.0
38684,7/15/15,NaN,NaN,Oscillibacter_sp_KLE_1745,0.0
38685,7/19/15,NaN,NaN,Oscillibacter_sp_KLE_1745,0.0


In [89]:
# CLEAN start (do not reuse the old 'records')
records = []

# use the species actually present in this sheet (no recomputing anything else)
species_present = [x for x in labels_clean.tolist()
                   if (x is not None) and (x in dict_speciesToPanels)]

# Date, Species, Abundance long format
for sp in species_present:
    s = pd.to_numeric(get_row_values(sp), errors="coerce")  # indexed by your existing col_date
    for d in col_date:
        v = s.get(d)
        if pd.notna(v):
            records.append((d, sp, float(v)))

df_longMicrobiome = pd.DataFrame(records, columns=["Date", "Species", "Abundance"])

# keep your original date order
df_longMicrobiome["Date"] = pd.Categorical(df_longMicrobiome["Date"],
                                           categories=col_date, ordered=True)
df_longMicrobiome = df_longMicrobiome.sort_values(["Date", "Species"]).reset_index(drop=True)

# map panels, dominant species, healthy average and max
df_longMicrobiome["Panel"] = df_longMicrobiome["Species"].map(dict_speciesToPanels)
df_longMicrobiome["DominantSpecies"] = df_longMicrobiome["Species"].map(dict_speciesToDominantSpecies16S)
df_longMicrobiome["HealthyAverage"] = df_longMicrobiome["Species"].map(dict_speciesToHealthyAverage)
df_longMicrobiome["HealthyMax"] = df_longMicrobiome["Species"].map(dict_speciesToHealthyMax)
# END CLEAN
df_longMicrobiome


,Date,Species,Abundance,Panel,DominantSpecies,HealthyAverage,HealthyMax
0,12/28/11,Adlercreutzia_equolifaciens,0.031,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
1,12/28/11,Aeromonas_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
2,12/28/11,Aggregatibacter_segnis,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
3,12/28/11,Aggregatibacter_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
4,12/28/11,Akkermansia_muciniphila,12.328,Xin Fang MetaphlAN 2 Metagenomics,NaN,0.984429,8.439
...,...,...,...,...,...,...,...
16138,8/11/19,p__Proteobacteria;f__Pasteurellaceae;g__Aggreg...,0.000,16S,None,NaN,NaN
16139,8/11/19,p__Proteobacteria;f__Pseudomonadaceae;g__Pseud...,0.060,16S,None,NaN,NaN
16140,8/11/19,p__Synergistetes;f__Synergistaceae;g__Synergistes,0.030,16S,None,NaN,NaN
16141,8/11/19,p__Tenericutes;f__;g__,0.580,16S,None,NaN,NaN


In [90]:
# === LONG + MAPPINGS (one go, only using what you already defined) ===
# uses: df_microbiome, labels_clean, col_date, get_row_values,
#       dict_speciesToPanels, dict_speciesToDominantSpecies16S,
#       dict_speciesToHealthyAverage, dict_speciesToHealthyMax

# 1) long rows: (Date, Species, Abundance) — skip missing, keep zeros
records = []
species_present = [x for x in labels_clean.tolist()
                   if (x is not None) and (x in dict_speciesToPanels)]

for sp in species_present:
    s = pd.to_numeric(get_row_values(sp), errors="coerce")  # indexed by your existing col_date
    for d in col_date:
        v = s.get(d)
        if pd.notna(v):
            records.append((d, sp, float(v)))

df_longMicrobiome = pd.DataFrame(records, columns=["Date", "Species", "Abundance"])

# 2) keep your original date order + sort
df_longMicrobiome["Date"] = pd.Categorical(df_longMicrobiome["Date"],
                                           categories=col_date, ordered=True)
df_longMicrobiome = df_longMicrobiome.sort_values(["Date","Species"]).reset_index(drop=True)

# 3) add mappings (panel, dominant species, healthy refs)
df_longMicrobiome["Panel"]           = df_longMicrobiome["Species"].map(dict_speciesToPanels)
df_longMicrobiome["DominantSpecies"] = df_longMicrobiome["Species"].map(dict_speciesToDominantSpecies16S)
df_longMicrobiome["HealthyAverage"]  = df_longMicrobiome["Species"].map(dict_speciesToHealthyAverage)
df_longMicrobiome["HealthyMax"]      = df_longMicrobiome["Species"].map(dict_speciesToHealthyMax)

df_longMicrobiome

,Date,Species,Abundance,Panel,DominantSpecies,HealthyAverage,HealthyMax
0,12/28/11,Adlercreutzia_equolifaciens,0.031,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
1,12/28/11,Aeromonas_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
2,12/28/11,Aggregatibacter_segnis,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
3,12/28/11,Aggregatibacter_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
4,12/28/11,Akkermansia_muciniphila,12.328,Xin Fang MetaphlAN 2 Metagenomics,NaN,0.984429,8.439
...,...,...,...,...,...,...,...
16138,8/11/19,p__Proteobacteria;f__Pasteurellaceae;g__Aggreg...,0.000,16S,None,NaN,NaN
16139,8/11/19,p__Proteobacteria;f__Pseudomonadaceae;g__Pseud...,0.060,16S,None,NaN,NaN
16140,8/11/19,p__Synergistetes;f__Synergistaceae;g__Synergistes,0.030,16S,None,NaN,NaN
16141,8/11/19,p__Tenericutes;f__;g__,0.580,16S,None,NaN,NaN


In [91]:
# Complerness check of abundance values
df_longMicrobiome['Abundance'].isnull().sum()

0

In [92]:
# Code to check for duplicates
dups = (
    df_longMicrobiome[df_longMicrobiome.duplicated(["Date","Species"], keep=False)]
    .sort_values(["Date","Species"])
)
dups

,Date,Species,Abundance,Panel,DominantSpecies,HealthyAverage,HealthyMax
190,12/28/11,p__Firmicutes;f__;g__,9.52,16S,Clostridium_sp_L2_50,NaN,NaN
191,12/28/11,p__Firmicutes;f__;g__,9.52,16S,Clostridium_sp_L2_50,NaN,NaN
197,12/28/11,p__Firmicutes;f__Lachnospiraceae;g__,4.83,16S,None,NaN,NaN
198,12/28/11,p__Firmicutes;f__Lachnospiraceae;g__,4.83,16S,None,NaN,NaN
421,4/3/12,p__Firmicutes;f__;g__,2.23,16S,Clostridium_sp_L2_50,NaN,NaN
...,...,...,...,...,...,...,...
16058,7/28/19,p__Firmicutes;f__Lachnospiraceae;g__,6.90,16S,None,NaN,NaN
16102,8/11/19,p__Firmicutes;f__;g__,9.82,16S,Clostridium_sp_L2_50,NaN,NaN
16103,8/11/19,p__Firmicutes;f__;g__,9.82,16S,Clostridium_sp_L2_50,NaN,NaN
16109,8/11/19,p__Firmicutes;f__Lachnospiraceae;g__,9.44,16S,None,NaN,NaN


In [93]:


dups = df_longMicrobiome[df_longMicrobiome.duplicated(['Date','Species'], keep=False)].copy()
dups['Abundance'] = pd.to_numeric(dups['Abundance'], errors='coerce')

all_same = dups.groupby(['Date','Species'])['Abundance'].nunique(dropna=True).le(1).all()
print(all_same)  # True => all the same, False => at least one group differs

True


/var/folders/nf/dp5pbtr55rn8mww76lsrkt800000gn/T/ipykernel_80014/3982303624.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  all_same = dups.groupby(['Date','Species'])['Abundance'].nunique(dropna=True).le(1).all()


In [94]:
# remove duplicate (Date, Species) rows (values are identical)
n_before = len(df_longMicrobiome)

df_longMicrobiome = (
    df_longMicrobiome
        .sort_values(["Date", "Species"])
        .drop_duplicates(subset=["Date", "Species"], keep="first")
        .reset_index(drop=True)
)

print(f"Removed {n_before - len(df_longMicrobiome)} duplicate rows.")
# sanity check
assert not df_longMicrobiome.duplicated(["Date","Species"]).any()

Removed 318 duplicate rows.


In [95]:
df_longMicrobiome

,Date,Species,Abundance,Panel,DominantSpecies,HealthyAverage,HealthyMax
0,12/28/11,Adlercreutzia_equolifaciens,0.031,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
1,12/28/11,Aeromonas_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
2,12/28/11,Aggregatibacter_segnis,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
3,12/28/11,Aggregatibacter_unclassified,0.000,Xin Fang MetaphlAN 2 Metagenomics,NaN,NaN,NaN
4,12/28/11,Akkermansia_muciniphila,12.328,Xin Fang MetaphlAN 2 Metagenomics,NaN,0.984429,8.439
...,...,...,...,...,...,...,...
15820,8/11/19,p__Proteobacteria;f__Pasteurellaceae;g__Aggreg...,0.000,16S,None,NaN,NaN
15821,8/11/19,p__Proteobacteria;f__Pseudomonadaceae;g__Pseud...,0.060,16S,None,NaN,NaN
15822,8/11/19,p__Synergistetes;f__Synergistaceae;g__Synergistes,0.030,16S,None,NaN,NaN
15823,8/11/19,p__Tenericutes;f__;g__,0.580,16S,None,NaN,NaN


In [96]:
# Write out the long format file
df_longMicrobiome.to_csv(fp_microbiomeLong, index=False)

## Create Gut Microbiome Wide DataFrame

In [97]:
# Create wide format by pivoting
df_wideMicrobiome = df_longMicrobiome.pivot(index="Species", columns="Date", values="Abundance").reset_index()
df_wideMicrobiome

Date,Species,12/28/11,4/3/12,8/7/12,11/6/12,1/26/13,2/8/13,2/5/13,3/24/13,4/7/13,...,4/14/19,4/28/19,5/12/19,5/19/19,6/2/19,6/16/19,6/23/19,7/14/19,7/28/19,8/11/19
0,Adlercreutzia_equolifaciens,0.031,0.035,0.143,0.064,0.041,0.036,NaN,0.008,0.006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aeromonas_unclassified,0.000,0.000,0.000,0.000,0.000,0.000,NaN,0.000,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aggregatibacter_segnis,0.000,0.000,0.000,0.000,0.000,0.014,NaN,0.015,0.562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aggregatibacter_unclassified,0.000,0.000,0.000,0.000,0.000,0.000,NaN,0.000,0.137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Akkermansia_muciniphila,12.328,0.950,3.842,5.579,3.518,3.212,NaN,1.063,0.892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,p__Proteobacteria;f__Pasteurellaceae;g__Aggreg...,0.000,0.000,0.000,0.000,0.000,0.040,0.29,0.030,1.030,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
225,p__Proteobacteria;f__Pseudomonadaceae;g__Pseud...,0.000,0.010,0.000,0.010,0.010,0.010,0.01,0.010,0.030,...,0.02,0.00,0.10,0.10,0.33,0.09,0.00,0.09,0.08,0.06
226,p__Synergistetes;f__Synergistaceae;g__Synergistes,0.100,0.000,0.020,0.050,0.110,0.040,0.12,0.010,0.020,...,0.07,0.03,0.06,0.01,0.00,0.00,0.03,0.05,0.11,0.03
227,p__Tenericutes;f__;g__,0.990,0.000,0.010,0.030,0.160,0.110,0.05,0.350,0.160,...,0.64,0.50,0.22,0.33,0.39,0.39,0.40,0.27,0.16,0.58


In [98]:
# Write the wide format file
df_wideMicrobiome.to_csv(fp_microbiomeWide, index=False)